In [2]:
from qdrant_haystack import QdrantDocumentStore
import numpy as np

# Qdrant Vector Store

This is an example of using the [Qdrant](https://qdrant.tech/) vector store with fastRAG. This is done using the dependency `qdrant_haystack` and `qdrant_client` python connector. We assume you have a running server, e.g. by calling `docker run -p 6333:6333 qdrant/qdrant` locally. 

Two important settings are the dimension of the vectors and HNSW parameters. Qdrant uses HNSW index for faster search, with a tradeoff between accuracy and latency. In general, higher numbers mean better accuracy, lower latency and larger RAM usage. 

The parameters are specified when connecting the server and creating a new index; these cannot be changed after the index was created. 

In [3]:
dim = 100
index_name = "test_hnsw"

## Creating an Index

Need to specify the location of the Qdrant service, vector dimension, index name, similarity metric and optionally the HNSW configuration. 

In [4]:
q = QdrantDocumentStore(url="http://localhost:6333",
                        embedding_dim=dim,
                        timeout=60,
                        index=index_name,
                        embedding_field="embedding",
                        hnsw_config={"m": 128, "ef_construct": 100},
                        similarity='dot_product',
                        recreate_index=True)

## Insertion and Searching of Documents

We'll create a few documents; they must have an `id`, `content` and `embedding` keys but could contain more data such as text titles. 

In [5]:
docs = [{"id": 1, "content": "I like to go to the beach", "embedding": np.ones(dim)},
        {"id": 2, "content": "Where is my hat?", "embedding": np.ones(dim) * 2},
        {"id": 3, "content": "GPT4 is very nice", "embedding": np.ones(dim) * 3},]

Writing the documents to index with batching; deduplication of documents is on by default. 

In [6]:
q.write_documents(docs, index_name, batch_size=500)

500it [00:00, 598.14it/s]                                                                                                                                                                                                        


In [7]:
q.get_document_count(index=index_name)

3

## Query by embedding
Need to provide a vector and `top_k` value. In general can also query by text search which we won't show here. 

In [8]:
q.query_by_embedding(np.ones(dim), top_k=1, index=index_name)

[<Document: {'content': 'GPT4 is very nice', 'content_type': 'text', 'score': 0.9525741268224334, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': '<embedding of shape (100,)>', 'id': '3'}>]